In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用图像分类的 Vertex AI 模型花园 MediaPipe

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_mediapipe_image_classification.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>

  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_mediapipe_image_classification.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_mediapipe_image_classification.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"> 在 Vertex AI Workbench 中打开
    </a>
  </td>
</table>

**注意**: 这个笔记本已在以下环境中进行了测试：

* Python版本 = 3.9

**注意**: 在此Colab中链接的检查点和数据集不是由谷歌拥有或分发的，而是由第三方提供。在使用检查点和数据之前，请先查阅第三方提供的条款和条件。

## 概述

本笔记本演示了如何使用[MediaPipe Model Maker](https://developers.google.com/mediapipe/solutions/model_maker)在Vertex AI模型花园中训练一个设备上的图像分类模型。

### 目标

* 训练新模型
  * 将输入数据转换为训练格式
  * 创建[自定义作业](https://cloud.google.com/vertex-ai/docs/training/create-custom-job)来训练新模型
  * 导出模型

* 清理资源

### 成本

本教程使用Google Cloud的收费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI价格](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage价格](https://cloud.google.com/storage/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)根据您的预期使用情况生成成本估算。

## 在开始之前

只有在Colab上运行时才能运行以下命令以安装依赖项并与Google Cloud进行身份验证。

In [ ]:
! pip3 install --upgrade pip

import sys

if "google.colab" in sys.modules:
    ! pip3 install --upgrade google-cloud-aiplatform

    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

    from google.colab import auth as google_auth

    google_auth.authenticate_user()

将您的项目ID设置为####

**如果您不知道您的项目ID**，请查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}
REGION_PREFIX = REGION.split("-")[0]
assert REGION_PREFIX in (
    "us",
    "europe",
    "asia",
), f'{REGION} is not supported. It must be prefixed by "us", "asia", or "europe".'

### 创建一个云存储桶

创建一个存储桶来存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有在您的存储桶不存在时才能运行以下单元格来创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

### 导入库

In [ ]:
import json
import os
from datetime import datetime

import tensorflow
from google.cloud import aiplatform

### 初始化顶点 AI SDK 用于 Python

为您的项目初始化顶点 AI SDK 用于 Python。

In [ ]:
now = datetime.now().strftime("%Y%m%d-%H%M%S")

STAGING_BUCKET = os.path.join(BUCKET_URI, "temp/%s" % now)

EVALUATION_RESULT_OUTPUT_DIRECTORY = os.path.join(STAGING_BUCKET, "evaluation")
EVALUATION_RESULT_OUTPUT_FILE = os.path.join(
    EVALUATION_RESULT_OUTPUT_DIRECTORY, "evaluation.json"
)

EXPORTED_MODEL_OUTPUT_DIRECTORY = os.path.join(STAGING_BUCKET, "model")
EXPORTED_MODEL_OUTPUT_FILE = os.path.join(
    EXPORTED_MODEL_OUTPUT_DIRECTORY, "model.tflite"
)

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

定义训练设备规格

In [ ]:
TRAINING_JOB_DISPLAY_NAME = "mediapipe_image_classifier_%s" % now
TRAINING_CONTAINER = f"{REGION_PREFIX}-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/mediapipe-train"
TRAINING_MACHINE_TYPE = "n1-highmem-16"
TRAINING_ACCELERATOR_TYPE = "NVIDIA_TESLA_V100"
TRAINING_ACCELERATOR_COUNT = 2

训练您定制的模型

### 为训练准备输入数据

对图像分类模型进行微调需要一个包含您希望完成模型能够识别的所有类别的数据集。您可以通过从公共数据集中仅保留与您的用例相关的类别、编制您自己的数据集，或两者结合的方式来实现这一点。数据集可以比从头开始训练新模型所需的数据集要小得多。例如，用于训练许多参考模型的 ImageNet 数据集包含数百万张图像，具有数千个类别。使用 Model Maker 进行迁移学习可以通过较小的数据集微调现有模型，并根据您的推断准确性目标表现良好。

您可以重新使用现有数据集，例如 `gs://cloud-samples-data-us-central1/vision/automl_classification/flowers` 来微调模型，或者将您自己的数据集上传到 GCS。如果您使用自己的数据集，请确保您的图像目录包含多个子目录，每个子目录对应特定的类别标签。您的训练数据也应遵循这种模式：<image_path>/<label_name>/<image_names>。*。

In [ ]:
training_data_path = "gs://cloud-samples-data-us-central1/vision/automl_classification/flowers"  # @param {type:"string"}
split_ratio = "0.8,0.1,0.1"  # @param {type:"string"}

### 设置微调选项

您可以在不同的模型架构之间进行选择，以进一步定制您的训练：

* MobileNet-V2
* EfficientNet-Lite0
* EfficientNet-Lite2
* EfficientNet-Lite4

要设置模型架构和其他训练参数，请调整以下数值：

In [ ]:
model_architecture = "mobilenet_v2"  # @param ["mobilenet_v2", "efficientnet_lite0", "efficientnet_lite2", "efficientnet_lite4"]

# The learning rate to use for gradient descent training.
learning_rate: float = 0.01  # @param {type:"number"}
# Batch size for training.
batch_size: int = 2  # @param {type:"number"}
# Number of training iterations over the dataset.
epochs: int = 10  # @param {type:"slider", min:0, max:100, step:1}
# If true, the base module is trained together with the classification layer on
# top.
do_fine_tuning: bool = False  # @param {type:"boolean"}
# A regularizer that applies a L1 regularization penalty.
l1_regularizer: float = 0.0  # @param {type:"number"}
# A regularizer that applies a L2 regularization penalty.
l2_regularizer: float = 0.0001  # @param {type:"number"}
# Amount of label smoothing to apply. See tf.keras.losses for more details.
label_smoothing: float = 0.1  # @param {type:"number"}
# A boolean controlling whether the training dataset is augmented by randomly
# distorting input images, including random cropping, flipping, etc. See
# utils.image_preprocessing documentation for details.
do_data_augmentation: bool = True  # @param {type:"boolean"}
# Number of training samples used to calculate the decay steps
# and create the training optimizer.
decay_samples: int = 2560000  # @param {type:"number"}
# Number of warmup steps for a linear increasing warmup schedule on learning
# rate. Used to set up warmup schedule by model_util.WarmUp.
warmup_epochs: int = 2  # @param {type:"number"}

### 运行微调

准备好您的训练数据集和微调选项后，您就可以开始微调过程了。这个过程需要大量资源，根据您可用的计算资源，可能需要几分钟到几个小时的时间。在使用GPU处理的Vertex AI上，下面的示例微调大约需要4-6分钟来训练约3700张图像。

要开始微调过程，请使用以下代码：

In [ ]:
model_export_path = EXPORTED_MODEL_OUTPUT_DIRECTORY
evaluation_result_path = EVALUATION_RESULT_OUTPUT_DIRECTORY

worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": TRAINING_MACHINE_TYPE,
            "accelerator_type": TRAINING_ACCELERATOR_TYPE,
            "accelerator_count": TRAINING_ACCELERATOR_COUNT,
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": TRAINING_CONTAINER,
            "command": [],
            "args": [
                "--task_name=image_classifier",
                "--training_data_path=%s" % training_data_path,
                "--model_export_path=%s" % model_export_path,
                "--evaluation_result_path=%s" % evaluation_result_path,
                "--split_ratio=%s" % split_ratio,
                "--model_architecture=%s" % model_architecture,
                "--hparams=%s"
                % json.dumps(
                    {
                        "learning_rate": learning_rate,
                        "batch_size": batch_size,
                        "epochs": epochs,
                        "do_fine_tuning": do_fine_tuning,
                        "l1_regularizer": l1_regularizer,
                        "l2_regularizer": l2_regularizer,
                        "label_smoothing": label_smoothing,
                        "do_data_augmentation": do_data_augmentation,
                        "decay_samples": decay_samples,
                        "warmup_epochs": warmup_epochs,
                    }
                ),
            ],
        },
    }
]

training_job = aiplatform.CustomJob(
    display_name=TRAINING_JOB_DISPLAY_NAME,
    project=PROJECT_ID,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=STAGING_BUCKET,
)

training_job.run()

## 评估并导出模型

### 评估性能

在对模型进行微调后，我们将在测试数据集上评估训练结果，这通常是原始数据集中未在训练过程中使用的一部分。一般认为，准确度在0.8和0.9之间是非常好的，但您的使用情况要求可能有所不同。您还应考虑模型产生推断的速度。更高的准确度通常会以较长的推断时间为代价。

In [ ]:
def get_evaluation_result(evaluation_result_path):
    try:
        with tensorflow.io.gfile.GFile(evaluation_result_path, "r") as input_file:
            evalutation_result = json.loads(input_file.read())
        return evalutation_result["accuracy"], evalutation_result["loss"]
    except:
        print(
            "Evaluation result not found. Your test dataset is likely "
            + "empty. You can adjust the size of your test dataset or adjust "
            + "how you split your dataset."
        )
        return None


evaluation_result = get_evaluation_result(EVALUATION_RESULT_OUTPUT_FILE)

if evaluation_result is not None:
    print("Accuracy:", evaluation_result[0])
    print("Loss:", evaluation_result[1])

### 导出模型
在微调和评估模型后，您可以保存Tensorflow Lite模型，通过在MediaPipe Studio的[图像分类](https://mediapipe-studio.webapps.google.com/demo/image_classifier)演示中尝试它，或者通过按照[图像分类任务指南](https://developers.google.com/mediapipe/solutions/vision/image_classifier)将其集成到您的设备上的应用程序中。导出的模型包含所需的模型元数据生成，以及一个分类标签文件。

In [ ]:
import sys

def copy_model(model_source, model_dest):
    ! gsutil cp {model_source} {model_dest}

copy_model(EXPORTED_MODEL_OUTPUT_FILE, "image_classification_model.tflite")

if "google.colab" in sys.modules:
    from google.colab import files

    files.download("image_classification_model.tflite")

清理

In [ ]:
# Delete training data and jobs.
if training_job.list(filter=f'display_name="{TRAINING_JOB_DISPLAY_NAME}"'):
    training_job.delete()

!gsutil rm -r {STAGING_BUCKET}